In [2]:
import networkx as nx
import pandas as pd
import ast
from pyvis.network import Network
import numpy as np
from chord import Chord

In [3]:
livros_df=pd.read_csv('livros_df.csv',index_col=0)

In [4]:
gen =livros_df['Genero'].unique()
cores=['#FF0000','#00FFFF','#00FF00','#FF00FF']
cor_dict={g:c for g,c in zip(gen,cores)}
#Função para criar a network
def criar_network(df):
    grafo = nx.DiGraph()
    livros = df['Titulo']
    reviewers_dict = df['Reviewer']
    generos=df['Genero']
    for livro, (reviewers,genero) in zip(livros,zip(reviewers_dict,generos)):
        reviewers = ast.literal_eval(reviewers)
        #Defindindo nodes
        node_livro =(livro,{'tipo':'Livro','color':cor_dict[genero],'size':30})
        reviewer_lst=[(value,{'tipo':'reviewer','id':key,'color':'orange','shape':'triangle','size':30}) for key,value in reviewers.items()]
        #Definindo conexões
        conexoes = [(node_livro[0],r[0]) for r in reviewer_lst]
        #Criando a network
        grafo.add_nodes_from([node_livro])
        grafo.add_nodes_from(reviewer_lst)
        grafo.add_edges_from(conexoes,color='#f1dba1')
    return grafo

Criando grafo

In [5]:
grafo = criar_network(livros_df)
#Definindo as autoridades(reviewers) e hubs(livros)
hubs_dict, auts_dict = nx.hits(grafo)
hubs_lst=[(k,v)for k,v in hubs_dict.items()]
aut_lst=[(k,v)for k,v in auts_dict.items()]
hubs_sort =sorted(hubs_lst,key=lambda x: x[1],reverse=True)[:20]
aut_sort =sorted(aut_lst,key=lambda x: x[1],reverse=True)[:10]
hubs=[hub[0] for hub in hubs_sort]
auts=[auts[0] for auts in aut_sort]

Explorando a network

In [22]:
for hub in hubs[:10]:
    print(f"{hub}:{grafo.degree(hub)} reviews")


      Rules of Civility
:273 reviews

      The Great Alone
:282 reviews

      The Immortalists
:223 reviews

      The Heart's Invisible Furies
:282 reviews

      The Library Book
:290 reviews

      The Firebird
:267 reviews

      Lilac Girls
:272 reviews

      The Alice Network
:289 reviews

      The Dutch House
:280 reviews

      The Paris Wife
:278 reviews


In [21]:
for aut in auts:
    print(f"{aut}: {grafo.degree(aut)} reviews")

Sarah: 425 reviews
Jennifer: 379 reviews
Lisa: 362 reviews
Laura: 334 reviews
Amy: 313 reviews
Michelle: 318 reviews
Julie: 289 reviews
Katie: 282 reviews
Emily: 277 reviews
Amanda: 277 reviews


Criando subgrafo com apenas as autoridade e hubs reduzindo o tamanho da network para gerar a visualização

In [17]:
aut_hub=set(auts+hubs)
aut_hub_grafo = grafo.subgraph(aut_hub)

Criando visualização da network

In [18]:

network = Network(height='80%',width='80%',bgcolor='#222222',font_color='white')
network.from_nx(aut_hub_grafo)
network.show_buttons(filter_=['physics'])
network.barnes_hut()
network.show('grafo_aut_hub2.html')

Preparando os dados para gráfico de conexões entre os géneros

In [15]:
generos = livros_df['Genero'].unique()
Fantasy_dic={}
Horror_dic={}
Hist_Fiction_dic={}
Non_Fiction_dic={}
dic_dics={'Fantasy':Fantasy_dic,'Horror':Horror_dic,'Historical Fiction':Hist_Fiction_dic,'Non Fiction':Non_Fiction_dic}
#Função para agrupar os dicionarios de reviewers de cada gênero
def agrupar_reviewers(dic_dics):
    for genero in generos:
        dic_series = livros_df.Reviewer[livros_df['Genero']==genero].to_list()
        for dic in dic_series:
            dic= ast.literal_eval(dic)
            dic_dics[genero].update(dic)
    return dic_dics

#Função para calcular quantos reviewer de um genero avaliaram livros de outro genero
def conexao_generos(dic,genero):
    lista=[]
    genero_set = set(dic[genero].keys())
    chaves = list(dic.keys())
    for chave in chaves:
        if chave == genero:
            #A interseção seria todo o conjunto, mas para o grafico este valor deve ser 0
            lista.append(0)
        else:
            dic_set=set(dic[chave].keys())
            #Agrupar os reviewers que avaliaram livros do genero em questão com os outros generos
            intersecao = genero_set.intersection(dic_set)
            lista.append(len(intersecao))
    return lista

In [36]:
grupo_reviewers = agrupar_reviewers(dic_dics)
fantasy_con=conexao_generos(grupo_reviewers,'Fantasy')
horror_con=conexao_generos(grupo_reviewers,'Horror')
hist_fiction_con=conexao_generos(grupo_reviewers,'Historical Fiction')
non_fiction_con=conexao_generos(grupo_reviewers,'Non Fiction') 
#Criando matriz
chord_data=[fantasy_con,horror_con,hist_fiction_con,non_fiction_con]

Criando Gráfico

In [45]:
Chord(chord_data,list(generos)).to_html('conexao_generos.html')

In [7]:
grafo.number_of_nodes()

50226